In [1]:
%%bash
hostname

piano.ucsf.edu


In [2]:
from collections import namedtuple
import jax
import jax.numpy as jnp
import jax.scipy as jsp
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
import pandas as pd
from pathlib import Path
from functools import partial
import itertools
from itertools import combinations
import re
import requests
import json
import scipy as sp
import scipy.stats
import sys
import time
import pyext.src.pynet_rng as rng
import timeit
import pyext.src.matrix as mat
import pyext.src.stats as stats

from src.wishart_synthetic_benchmark import (
    ccscatter,
    check_cov,
    df_from_stats,
    get_precision_matrix_stats,
    get_prior_pred,
    helper_vline_hist,
    margins_plot,
    quad_plot,
    randPOSDEFMAT,
    rprior,
    rprior_pred,
    sample_from_prior,
    scatter_plot,
    simulate_from_prior,
    try_sampling,
    ground_truth_pair_plot
)

from src.cullin_benchmark_test import (
    CullinBenchMark,
    accumulate_indicies,
    bar_plot_df_summary,
    binary_search,
    biogrid_df_report,
    check_biogrid_data,
    check_bounds,
    compare_reports,
    find_bounds,
    find_end,
    find_start,
    format_biogrid_df_report,
    get_all_indicies,
    get_biogrid_summary,
    get_json_report_from_report,
    make_bounds,
    show_idmapping_results,
    transform_and_validate_biogrid,
    uniprot_id_mapping,
    get_experimental_coverage_df,
    triangular_to_symmetric,
    coverage_plot,
    transform_cullin_benchmark_data,
    split,
    to_entrez,
    get_bounds_from_id_mapping
)

In [3]:
# Flags

GET_CULLIN_BG = True

In [4]:
# Global helper functions

inv = sp.linalg.inv

In [5]:
# Global Plotting params

divergent = "seismic"
sequential = "Reds"

In [6]:
%%bash
cd ../data/raw/biogrid/ && shasum -c checksum512.txt

BIOGRID-ALL-4.4.206.tab3.txt: OK


In [7]:
check_biogrid_data()

Checking biogrid data for missing tabs
Passed
n-lines 2312699


In [9]:
biogrid = pd.read_csv("../data/raw/biogrid/BIOGRID-ALL-4.4.206.tab3.txt", delimiter="\t")

/tmp/ipykernel_2564433/2573753576.py:1: DtypeWarning: Columns (1,2,18) have mixed types. Specify dtype option on import or set low_memory=False.
  biogrid = pd.read_csv("../data/raw/biogrid/BIOGRID-ALL-4.4.206.tab3.txt", delimiter="\t")


In [10]:
biogrid = transform_and_validate_biogrid(biogrid)

Processing Entrez Gene Interactor A
Processing Entrez Gene Interactor B
Change dtypes to categorical
Select columns


In [17]:
# Load in the Cullin Data
cullin_benchmark = CullinBenchMark(dirpath=Path("../data/raw/cullin_e3_ligase"))
#cullin_benchmark.load_data()

In [20]:
# Do Uniprot ID mapping
id_mapping, failed_ids, prey_set_idmapping_input = uniprot_id_mapping(cullin_benchmark)

from UniProtKB_AC-ID to GeneID size 5000
POST https://rest.uniprot.org/idmapping/run
          respone  200
          jobId 509258eae75bd64606325ddf25fc85ec20daf815
Waiting 15 s
GET https://rest.uniprot.org/idmapping/status/509258eae75bd64606325ddf25fc85ec20daf815
GET https://rest.uniprot.org/idmapping/stream/509258eae75bd64606325ddf25fc85ec20daf815


In [21]:
show_idmapping_results(id_mapping, failed_ids, prey_set_idmapping_input)

12 failed to map
2835 succeeded
of 2847 total


In [46]:
# Optionally write the ID mapping to a file and update the reference
WRITE_IDMAPPING=False
if WRITE_IDMAPPING:
    pd.DataFrame(data={'UKBID': list(id_mapping.keys()), 'GID': list(id_mapping.values())}).to_csv(
        '../data/interim/cullin_e3_ligase/id_mapping.csv',
    index=False)
    
    pd.DataFrame(data={'Variable': ['UKBID', 'GID'], 
                       'Variable Name': ['Uniprot Knowledge Base Accession ID', 'Entrez Gene ID'],
                      'Measurement Unit': ['Unique Alphanumeric ID', 'Unique Numeric ID'],
                      }).to_csv('../references/data/interim/cullin_e3_ligase/id_mapping.csv',
                               index=False)

In [53]:
%%bash
ls ../data/raw/cullin_e3_ligase

1-s2.0-S1931312819302537-mmc2.xlsx
1-s2.0-S1931312819302537-mmc3.xlsx
1-s2.0-S1931312819302537-mmc4.xlsx
1-s2.0-S1931312819302537-mmc5.xlsx
ground_truth
PRIDE
summary.json
test
train


In [ ]:
# Make Cullin Reference Guide



In [51]:
pd.read_excel("../data/raw/cullin_e3_ligase/1-s2.0-S1931312819302537-mmc2.xlsx")

,Bait,Prey,PreyGene,Spec,SpecSum,AvgSpec,NumReplicates,ctrlCounts,AvgP,MaxP,TopoAvgP,TopoMaxP,SaintScore,FoldChange,BFDR
0,CBFBwt_MG132,vifprotein,vifprotein,22|22|26|34,104,26.00,4,0|0|1|1|0|1|3|2|1|0|0|0,1.00,1.0,1.00,1.0,1.00,34.67,0.00
1,CBFBwt_MG132,Q9UBF6,RBX2_HUMAN,9|11|12|16,48,12.00,4,0|0|0|0|0|0|0|0|1|0|0|0,1.00,1.0,1.00,1.0,1.00,120.00,0.00
2,CBFBwt_MG132,Q9C0K0,BC11B_HUMAN,8|8|26|27,69,17.25,4,0|2|2|0|3|1|1|0|4|3|1|1,0.96,1.0,0.96,1.0,0.96,11.50,0.00
3,CBFBwt_MG132,Q93034,CUL5_HUMAN,78|77|66|99,320,80.00,4,0|0|0|0|0|0|0|1|0|0|1|0,1.00,1.0,1.00,1.0,1.00,480.00,0.00
4,CBFBwt_MG132,Q8TEB1,DCA11_HUMAN,24|19|15|11,69,17.25,4,0|0|0|0|0|1|3|0|1|0|1|0,1.00,1.0,1.00,1.0,1.00,34.50,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6550,ELOBwt_MG132,O00232,PSD12_HUMAN,10|9|12|12,43,10.75,4,12|10|11|17|17|19|19|15|7|11|16|11,0.00,0.0,0.00,0.0,0.00,0.78,0.92
6551,ELOBwt_MG132,O00139,KIF2A_HUMAN,0|0|0|0,0,0.00,4,0|0|1|0|1|0|3|4|0|0|0|0,0.00,0.0,0.00,0.0,0.00,0.00,0.92
6552,ELOBwt_MG132,A8MTZ0,BBIP1_HUMAN,1|0|1|1,3,0.75,4,0|0|0|0|0|0|0|0|0|0|0|0,0.00,0.0,0.00,0.0,0.00,7.50,0.92
6553,ELOBwt_MG132,A4UGR9,XIRP2_HUMAN,0|0|0|0,0,0.00,4,0|0|0|0|0|0|0|0|0|1|0|0,0.00,0.0,0.00,0.0,0.00,0.00,0.92


In [ ]:
# Check the Failed Cases
failed_df = cullin_benchmark.data[cullin_benchmark.data['Prey'].apply(lambda x: x in failed_ids)]

# The failed cases amount to only 20 Bait prey Pairs
# The Saint Score < 0.14 for all cases
# Therefore we ignore the 13 failed cases instead of mapping them
# Except for L0R6Q1. Consider for later

In [ ]:
# Set the Values for the Entrez Gene Id Columns

eids = set(val for key, val in id_mapping.items())

colA = "Entrez Gene Interactor A"
colB = "Entrez Gene Interactor B"

col = colA
eids_in_biogrid = set(map(lambda x: x if int(x) in biogrid.loc[:, col] else None, eids))
eids_in_biogrid.remove(None)
bounds_A = make_bounds(biogrid, col, eids_in_biogrid)

col = colB
eids_in_biogrid = set(map(lambda x: x if int(x) in biogrid.loc[:, col] else None, eids))
eids_in_biogrid.remove(None)
bounds_B = make_bounds(biogrid, col, eids_in_biogrid)
            

check_bounds(biogrid, bounds_A, eids, colnum=1)
check_bounds(biogrid, bounds_B, eids, colnum=2)

In [ ]:
if GET_CULLIN_BG:
    index_labels = get_all_indicies(biogrid, bounds_A, bounds_B, 1, 2)

    # Look at the subset of biogrid

    cullin_bg = biogrid.loc[index_labels]

    # Validate the data frame
    assert np.all(cullin_bg.iloc[:, 1].apply(lambda x: True if str(int(x)) in eids_in_biogrid else False))
    assert np.all(cullin_bg.iloc[:, 2].apply(lambda x: True if str(int(x)) in eids_in_biogrid else False))

    nnodes = len(eids_in_biogrid)
    n_possible_edges = int(0.5*nnodes*(nnodes-1))

    nself = len(cullin_bg[cullin_bg.iloc[:, 1]==cullin_bg.iloc[:, 2]])
    not_self = cullin_bg.iloc[:, 1]!=cullin_bg.iloc[:, 2]

    cullin_bg = cullin_bg[not_self]

    #df = cullin_bg


    # How many edges are unique?

In [ ]:
cullin_report = biogrid_df_report(cullin_bg)

In [ ]:
# Annotate the cullin benchmark with entrez genes and rows and columns
cullin_benchmark_transformed = transform_cullin_benchmark_data(cullin_benchmark.data, id_mapping)

In [ ]:
# Define the training set
bait_name = "CBFBwt_MG132"
s = cullin_benchmark_transformed["Bait"] == bait_name
trainingset = cullin_benchmark_transformed[s]

saint_cutoff = 0.7
s = trainingset["SaintScore"] >= saint_cutoff

trainingset = trainingset[s]

In [ ]:
# Get the biogrid subset for the training set
training_set_id_mapping = {key:id_mapping[key] for key in trainingset.iloc[1:len(trainingset), 1]} # ignore vifprotein
s = False
training_set_eid_set= set(trainingset["Entrez"])
for key, entrezid in training_set_id_mapping.items():
    s1 = cullin_bg["Entrez Gene Interactor A"] == float(entrezid)
    s = s1 | s

# Select the union
for key, entrezid in training_set_id_mapping.items():
    s1 = cullin_bg["Entrez Gene Interactor B"] == float(entrezid)
    #s = s1 & s

    

# Superset of at least one interactor



assert len(s) == len(cullin_bg)

training_bg = cullin_bg[s]

In [ ]:
training_bg[training_bg[colA] == 9616.0]

In [ ]:
# Generate the report for the training data
trainingset_report = biogrid_df_report(cullin_bg)

In [ ]:
# Experimental Coverage
trainingset_bg_coverage = get_experimental_coverage_df(training_bg, trainingset_report)

In [ ]:
coverage_plot(trainingset_bg_coverage)

In [ ]:
training_bg

In [ ]:
# highlight the area of interaction using labels
# remove the '-'
try:
    training_set_eid_set.remove("-")
except KeyError:
    ...
training_set_eid_list = [float(i) for i in training_set_eid_set]
trainingset_bg_coverage = trainingset_bg_coverage.loc[training_set_eid_list, training_set_eid_list]



In [ ]:
w = 8
h = 8

plt.figure(figsize=(w, h))
plt.title(f"Experimental Coverage of {bait_name} prey in biogrid")
plt.imshow(trainingset_bg_coverage + trainingset_bg_coverage.T, vmin=0, cmap=sequential)
#xlabels = 
plt.show()

In [ ]:
cols = list(eid_to_preygene.keys())
cols = cols[1:len(cols)] # drop '-'
cols = [float(i) for i in cols]

scratch = training_bg_coverage.loc[cols, cols]

fig1 = np.ones((p, p)) * 1000
fig1[1:p, 1:p] = scratch
fig, axs = ground_truth_pair_plot((fig1 + fig1.T) // 2, 
                                  np.array(np.mean(exp.samples, axis=0)), 
                                 overwrite_diags=False,
                                 vmin1=0, vmax1=150,
                                 vmin2=-12.5, vmax2=12.5,
                                 cmap1=sequential, cmap2=divergent)



axs[0].set_title(f"N Biogrid annotated interaction pairs")
axs[1].set_title(f"Ensemble average value over {'{:,}'.format(n_samples)} replicates")

xlabels1 = trainingset_bg_coverage.columns
eid_to_preygene = {}
for i, row in trainingset.loc[:, ["PreyGene", "Entrez"]].iterrows():
    eid_to_preygene[row["Entrez"]] = row["PreyGene"]

xlabels = [eid_to_preygene[str(int(eid))] for eid in cols]



axs[0].set_xticks(ticks=np.arange(p), labels=["vifprotein"] + xlabels, rotation=-45)
axs[0].set_yticks(ticks=list(range(p)), labels=["vifprotein"] + xlabels, rotation=45)

axs[1].set_xticks(ticks=np.arange(p), labels=list(trainingset["PreyGene"]), rotation=-45)
axs[1].set_yticks(ticks=np.arange(p), labels=list(trainingset["PreyGene"]), rotation=45)
plt.show()

In [ ]:
["-"] + ["a", 1, 7]

In [ ]:
assert exp.samples.shape == (n_samples, 16, 16)
assert scratch.shape == (15, 15)
scratch2 = np.mean(exp.samples, axis=0)
scratch2 = scratch2[1:16, 1:16]
assert scratch2.shape == scratch.shape


plt.scatter(scratch2[np.tril_indices(15, k=-1)], scratch.values[np.tril_indices(15, k=-1)])
plt.ylabel("Biogrid Annotation Rate")
plt.xlabel("Average edge value")

In [ ]:


import matplotlib as mpl
with mpl.rc_context({"font.size": 18}):

    plt.figure(figsize=(8, 8))

    assert exp.samples.shape == (n_samples, 16, 16)
    assert scratch.shape == (15, 15)
    scratch2 = np.mean(exp.samples, axis=0)
    scratch2 = scratch2[1:16, 1:16]
    assert scratch2.shape == scratch.shape


    x = scratch2[np.tril_indices(15, k=-1)]
    y = scratch.values[np.tril_indices(15, k=-1)]
    plt.plot(x, y, "o",color="b")
    plt.ylabel("Biogrid Annotation Rate")
    plt.xlabel("Average pairwise value")
    
    r, pval = sp.stats.pearsonr(x, y)
    
    decimals = 2
    r = np.round(r, decimals)
    pval = np.round(pval, decimals)
    s = "\N{greek small letter rho}"
    s += f" {r}"
    s += f"\np-val {pval}"
    plt.text(-5.5, 120, s)
    
    #plt.xlabel("Ground Truth")

    plt.legend()
    
    plt.show()

In [ ]:
# Map Prey Gene names to columns

?sp.stats.pearsonr


In [ ]:
cols = list(eid_to_preygene.keys())
cols = cols[1:len(cols)] # drop '-'
cols = [float(i) for i in cols]

In [ ]:
training_bg_coverage.loc[cols, cols]

In [ ]:
eid_to_preygene

In [ ]:
training_bg_coverage = get_experimental_coverage_df(cullin_bg, cullin_report)

In [ ]:
# Get Biogrid for the training set

index_labels = get_all_indicies(biogrid, bounds_A, bounds_B, 1, 2)

# Look at the subset of biogrid

cullin_bg = biogrid.loc[index_labels]

# Validate the data frame
assert np.all(cullin_bg.iloc[:, 1].apply(lambda x: True if str(int(x)) in eids_in_biogrid else False))
assert np.all(cullin_bg.iloc[:, 2].apply(lambda x: True if str(int(x)) in eids_in_biogrid else False))

nnodes = len(eids_in_biogrid)
n_possible_edges = int(0.5*nnodes*(nnodes-1))

nself = len(cullin_bg[cullin_bg.iloc[:, 1]==cullin_bg.iloc[:, 2]])
not_self = cullin_bg.iloc[:, 1]!=cullin_bg.iloc[:, 2]

cullin_bg = cullin_bg[not_self]

In [ ]:
# Operate on
cols = ["r1", "r3", "r3", "r4"]

df = trainingset
U = df[cols] @ df[cols].T
p = len(U)


# log10 transformation(SC) + prior

# Do the sampling
V = inv(U / 4 + np.eye(p))
key = jax.random.PRNGKey(3721)
n_samples = 1000000
exp = sample_from_prior(key, 15, p, n_samples, V)

In [ ]:
# How many failed ids are in the training set?

len(set(failed_ids).intersection(df["Prey"]))
# Conclusion - all ids except vif mapped

In [ ]:
# Get the experimental coverage in biogrid

        
d = get_experimental_coverage_df(cullin_bg, cullin_report)

In [ ]:
# Annotate the cullin benchmark with entrez genes and rows and columns
cullin_benchmark_transformed = transform_cullin_benchmark_data(cullin_benchmark.data, id_mapping)

In [ ]:
coverage_plot(d, w=16, h=14,)

In [ ]:
ground_truth_pair_plot(np.log10(U / 16), inv(U + np.eye(p)),
                      vmin1=1, vmax1=4.5, vmin2=-1., vmax2=1.,
                      cmap1=sequential, cmap2=divergent, title1="log 10 correlations", title2="Precision + Prior")


In [ ]:
ground_truth_pair_plot(inv(U + np.eye(p)), inv(U/4 + np.eye(p)),
                      vmin1=-1.5, vmax1=1.5, vmin2=-1.5, vmax2=1.5,
                      cmap1=divergent, cmap2=divergent, title1="Precision + Prior", title2="Scaled Precision + Prior")

In [ ]:
# 4 replicate Plot

fig, axs = ground_truth_pair_plot(np.array(np.var(exp.samples, axis=0)), np.array(np.mean(exp.samples, axis=0)), 
                                 overwrite_diags=False,
                                 vmin1=0,
                                 vmin2=-12.5, vmax2=12.5,
                                 cmap1=sequential, cmap2=divergent)

axs[0].set_title(f"Ensemble average variance")
axs[1].set_title(f"Ensemble average value over {'{:,}'.format(n_samples)} replicates")
plt.show()

In [ ]:
cocrystal_sel = cullin_bg["Experimental System"] == "Co-crystal Structure"
cullin_bg_cocrystal = cullin_bg[cocrystal_sel]

In [ ]:
s = cullin_benchmark_transformed["Bait"] == bait_name
s2 = cullin_benchmark_transformed["SaintScore"] >= saint_cutoff
s3 = s & s2
assert np.alltrue(cullin_benchmark_transformed.loc[s3,cols] == df[cols])

In [ ]:
# Check the experimental coverage for the training set
trainingset = cullin_benchmark_transformed[s3]
trainingset_report = biogrid_df_report(cullin_bg)

In [ ]:
# Must select a subset of biogrid, then do the report

In [ ]:
trainingset_coverage = get_experimental_coverage_df(trainingset, cullin_benchmark_transformed)

coverage_plot(trainingset_coverage)

In [ ]:
?coverage_plot

In [ ]:
?biogrid_df_report

In [ ]:
bait_name

In [ ]:
s = cullin_benchmark_transformed["Bait"] == bait_name
s2 = cullin_benchmark_transformed["SaintScore"] >= saint_cutoff
s3 = s & s2
assert np.alltrue(cullin_benchmark_transformed.loc[s3,cols] == df[cols])

In [ ]:
set(cullin_bg["Experimental System"])

In [ ]:
fig, axs = ground_truth_pair_plot(V, V, 
                                 overwrite_diags=False,
                                 vmin1=0,
                                 vmin2=-1.5, vmax2=1.5,
                                 cmap1=sequential, cmap2=divergent)